In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.delete_all_components()

In [4]:
design.get_chip_z('Q_chip')

'12um'

In [3]:
design.chips['C_chip'].size['size_x']='8mm'
design.chips['C_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['size_x']='8mm'
design.chips['Q_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['center_z']='12um'

# Resonator length

In [16]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [18]:
resonator_length_2=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)-0.8+0.034+0.13
    resonator_length_2.append(a)
print(resonator_length_2)


[7.817254683870284, 7.588788339841421, 7.37234654020913]


# Flipchip

In [6]:
#padgap 20->40 qubit fq 200MHz오르길

cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip',
                                            pad_gap='40um'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-60}um",
                                              pos_y=f"{q0y+75}um",
                                              pad_width="110um",
                                              pad_height="257um",
                                              cpw_stub_height="200um",
                                              orientation='270',                                            
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [7]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-3000um',
                                         pos_y = '2000um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '3000um',
                                         pos_y = '2000um',
                                         orientation='180',
                                         lead_length = '30um'))

In [8]:
clt0 = CoupledLineTee(design, "clt0", 
                        options=dict(pos_x='-2000um',
                                    pos_y='2000um', 
                                    coupling_length='300um',
                                    chip="C_chip",
                                    down_length='300um',
                                    fillet='99um',
                                    coupling_space='5um',
                                    mirror=True,
                                    open_termination=True))



In [9]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
fl1=RouteStraight(design, 'fl1',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [10]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [11]:
rr0=connect('rr0', 'clt0', 'second_end', 'c0', 'in',
            resonator_length_2[1],jogs,jogs, 
            '400um','0um', '100um', '100um')

# EPR Analysis

In [12]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [13]:
hfss.start()

INFO 04:55PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:55PM [load_ansys_project]: 	Opened Ansys App
INFO 04:55PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 04:55PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project20
INFO 04:55PM [connect_design]: No active design found (or error getting active design).
INFO 04:55PM [connect]: 	 Connected to project "Project20". No design detected


True

In [14]:
hfss.activate_ansys_design("flip12um_V4", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','clt0','rr0','fl0','fl1'],
                    [('clt0','prime_end')],
                    box_plus_buffer=False)

04:56PM 02s WARNING [activate_ansys_design]: The design_name=flip12um_V4? was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 04:56PM [connect_design]: 	Opened active design
	Design:    flip12um_V4? [Solution type: Eigenmode]
WARNING 04:56PM [connect_setup]: 	No design setup detected.
WARNING 04:56PM [connect_setup]: 	Creating eigenmode default setup.
INFO 04:56PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [15]:
hfss.close()

True

# LOM Analysis

In [12]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [13]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.45',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.45',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [17]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 04:25PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpd3d8m701.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5450000000, Maxwell, 1, False
INFO 04:25PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpedhot54j.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5450000000, Maxwell, 1, False
INFO 04:25PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmps453yd0f.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5450000000, Maxwell, 2, False
INFO 04:25PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpahgis_er.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5450000000, Maxwell, 3, False
INFO 04:25PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp4jnq8gkv.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5450000000, Maxwell, 4, False
INFO 04:25

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,354.110,-93.9440,-71.660,-127.0000
pad_bot_Q0,-93.944,121.2000,-25.644,-0.3323
pad_top_Q0,-71.660,-25.6440,120.850,-22.3680
quad_coupler_c0,-127.000,-0.3323,-22.368,150.2600


In [18]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 6.72
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.278291 [GHz]
EC 259.237552 [MHz]
EJ 14.854145 [GHz]
alpha -294.392917 [MHz]
dispersion 2.987730 [KHz]
Lq 10.995564 [nH]
Cq 74.719993 [fF]
T1 29.884368 [us]

**Coupling Properties**

tCqbus1 -11.032816 [fF]
gbus1_in_MHz -145.873686 [MHz]
χ_bus1 -5.094848 [MHz]
1/T1bus1 5325.692060 [Hz]
T1bus1 29.884368 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.548286,288.15543,14.854145,-330.085517,8.683365,[-90.88372752084703],[-3.1360440700611156],3.136044,90.883728
2,5.669114,301.659099,14.854145,-346.936649,13.53706,[-51.68229056366797],[-1.2741217374790739],1.274122,51.682291
3,5.341695,265.873727,14.854145,-302.536458,3.876365,[-182.6632009576488],[-8.856739262516793],8.856739,182.663201
4,5.355828,267.365869,14.854145,-304.371407,4.104512,[-171.44498324987177],[-7.986852762823175],7.986853,171.444983
5,5.321494,263.749182,14.854145,-299.926273,3.570204,[-163.1527591370539],[-6.8352800352651],6.835280,163.152759
6,5.317526,263.332986,14.854145,-299.415279,3.512706,[-152.25592161642282],[-5.914271763126088],5.914272,152.255922
7,5.305739,262.098775,14.854145,-297.900598,3.346817,[-150.63641523134064],[-5.679104424747084],5.679104,150.636415
8,5.28025,259.441193,14.854145,-294.642395,3.012151,[-147.9497893034042],[-5.257414230625011],5.257414,147.949789
9,5.276507,259.052158,14.854145,-294.165817,2.965644,[-147.7048860433256],[-5.2086331087799715],5.208633,147.704886
10,5.278291,259.237552,14.854145,-294.392917,2.98773,[-145.87368575771447],[-5.094848266037081],5.094848,145.873686


In [22]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
